In [1]:
import pandas as pd

In [39]:
hprc_biosample = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/hprc-production-biosample-table.tsv',sep='\t')
hprc_biosample.rename(columns={'Sample':'sample_id'},inplace=True)

In [36]:
hic_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/HIC_Y3_fillin_and_Y4_part1-redo/HIC_Y3_fillin_and_Y4_part1-redo-sample_file_hifi_qc_aggregate.csv')

In [37]:
hic_df.shape

(366, 5)

In [40]:
hic_df['sample_id'] = [sample_id.split('-')[0] for sample_id in hic_df['file_name'].tolist()]

In [41]:
hic_df['sample_id'].nunique()

62

In [42]:
hic_df = pd.merge(hic_df, hprc_biosample[hprc_biosample['sample_id'].isin(hic_df['sample_id'])], on='sample_id', how='inner')

In [43]:
hic_sra_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/HIC_Y3_fillin_and_Y4_part1-redo/sra_biosample_HIC_Y3_fillin_and_Y4_part1.csv')

In [44]:
# not included
hic_df[~hic_df['file_name'].isin(hic_sra_df['filename'])]

,file_name,total_bp,coverage,ntsm_score,ntsm_result,sample_id,Accession,familyID,Subpopulation,Superpopulation,Production Year
356,HG01940-1_S4_L004_R1_001.fastq.gz,92221589,0.03,0.978455,Similar,HG01940,SAMN37797101,PEL015,PEL,AMR,YR3
357,HG01940-1_S4_L004_R2_001.fastq.gz,92221589,0.03,0.979933,Similar,HG01940,SAMN37797101,PEL015,PEL,AMR,YR3
358,HG03611-2_FC2_S6_L002_R2_001.fastq.gz,151,0.00,1.000000,Similar,HG03611,SAMN41021628,NaN,BEB,SAS,YR4
359,HG03611-2_FC2_S6_L002_R1_001.fastq.gz,151,0.00,1.000000,Similar,HG03611,SAMN41021628,NaN,BEB,SAS,YR4
360,HG02015-2_S9_L004_R2_001.fastq.gz,3926,0.00,1.000000,Similar,HG02015,SAMN33621948,VN046,KHV,EAS,YR3
361,HG02015-2_S9_L004_R1_001.fastq.gz,3926,0.00,1.000000,Similar,HG02015,SAMN33621948,VN046,KHV,EAS,YR3
362,HG01252-1_S2_L002_R2_001.fastq.gz,453,0.00,1.000000,Similar,HG01252,SAMN37797098,CLM14,CLM,AMR,YR3
363,HG01252-1_S2_L002_R1_001.fastq.gz,453,0.00,1.000000,Similar,HG01252,SAMN37797098,CLM14,CLM,AMR,YR3
364,NA21110-2_FC1_S2_L002_R1_001.fastq.gz,151,0.00,1.000000,Similar,NA21110,SAMN41021624,NaN,GIH,SAS,YR4
365,NA21110-2_FC1_S2_L002_R2_001.fastq.gz,151,0.00,1.000000,Similar,NA21110,SAMN41021624,NaN,GIH,SAS,YR4


In [45]:
hic_sra_df = hic_sra_df[hic_sra_df['filename'].isin(hic_df['file_name'])]
hic_sra_df['common_id'] = hic_sra_df['filename'].str.extract(r'(^.+_L\d{3})')

In [49]:
df_R1 = hic_sra_df[hic_sra_df['filename'].str.contains('_R1_')].copy()
df_R2 = hic_sra_df[hic_sra_df['filename'].str.contains('_R2_')].copy()

In [64]:
hic_sra_merge_df = pd.merge(df_R1, df_R2, on='common_id', how='inner')

In [66]:
hic_sra_merge_df = hic_sra_merge_df.loc[:, ~hic_sra_merge_df.columns.str.endswith('_y')]
hic_sra_merge_df.columns = hic_sra_merge_df.columns.str.replace('_x$', '', regex=True)

In [72]:
hic_sra_merge_df['library_ID'] = hic_sra_merge_df['library_ID'].str.extract(r'(.+_L\d{3})')

In [76]:
hic_sra_merge_df.drop(columns=['common_id'],inplace=True)

In [77]:
hic_sra_merge_df.to_csv('sra_biosample_HIC_Y3_fillin_and_Y4_part1-redo.tsv',index=False, sep='\t')